# BeigeSage

This notebook uses the Beige Sage model to assign labels to all of the Beige Books chunks in the corpus.

### Reading in data

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import shutil

# Load the sentiment scores CSV
excel_path = r"C:/Users/MCOB PHD 14/Dropbox/Charlie's Dissertation/Beige Books/manual_sentiment.csv"
sentiment_data = pd.read_csv(excel_path)

# Define the label function
def label_sentiment(score):
    if score <= -0.3:
        return 0  # Negative
    elif score <= 0.2:
        return 1  # Mixed
    else:
        return 2  # Positive

# Define path where text files are stored
text_files_dir = r"C:/Users/MCOB PHD 14/Dropbox/Charlie's Dissertation/Beige Books/chunks"

# Load the text files and create a DataFrame
text_data = {}
for filename in os.listdir(text_files_dir):
    if filename.endswith('.txt'):
        file_path = os.path.join(text_files_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text_data[filename] = file.read()

text_df = pd.DataFrame(list(text_data.items()), columns=['file_names', 'text'])

### Load BeigeSage

In [6]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Define the path where the model and tokenizer are saved
saved_model_path = 'C:/Users/MCOB PHD 14/Desktop/bbFinal/Notebooks/BeigeSage'

# Load the saved tokenizer
tokenizer = RobertaTokenizer.from_pretrained(saved_model_path)

# Load the saved model
model = RobertaForSequenceClassification.from_pretrained(saved_model_path)

# Set the model to evaluation mode
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# Define a function to tokenize and predict sentiment
def predict_sentiment_with_status(texts):
    predictions = []
    total_texts = len(texts)
    for idx, text in enumerate(tqdm(texts, desc="Predicting Sentiment", ncols=100)):
        # Tokenize the text
        inputs = tokenizer(
            text,
            return_tensors="pt",       # Return as PyTorch tensors
            truncation=True,           # Truncate longer sequences
            padding='max_length',      # Pad to max length
            max_length=512             # Set maximum length
        )
        
        # Perform prediction
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Get the predicted label
        predicted_label = torch.argmax(outputs.logits, dim=1).item()
        predictions.append(predicted_label)
        
        # Print status update for every 100 texts
        if (idx + 1) % 100 == 0 or (idx + 1) == total_texts:
            print(f"Processed {idx + 1}/{total_texts} texts")

    return predictions

In [11]:
from tqdm import tqdm

# Apply the prediction function to the 'text' column of test_data DataFrame
test_texts = text_df['text'].tolist()  # Convert text column to list
text_df['predicted_label'] = predict_sentiment_with_status(test_texts)

# Step 4: Map the numerical labels back to the class names (optional)
label_map = {0: "Negative", 1: "Mixed", 2: "Positive"}
text_df['predicted_class'] = text_df['predicted_label'].map(label_map)

# Display the DataFrame with predictions
print(text_df[['text', 'predicted_label', 'predicted_class']])

Predicting Sentiment:  87%|████████████████████████████▋    | 25650/29521 [3:22:04<29:19,  2.20it/s]

Processed 25650/29521 texts


Predicting Sentiment:  87%|████████████████████████████▋    | 25660/29521 [3:22:09<28:12,  2.28it/s]

Processed 25660/29521 texts


Predicting Sentiment:  87%|████████████████████████████▋    | 25670/29521 [3:22:13<28:16,  2.27it/s]

...

Processed 29510/29521 texts


Predicting Sentiment: 100%|████████████████████████████████▉| 29520/29521 [3:52:52<00:00,  2.11it/s]

Processed 29520/29521 texts


Predicting Sentiment: 100%|█████████████████████████████████| 29521/29521 [3:52:52<00:00,  2.11it/s]

Processed 29521/29521 texts
                                                    text  predicted_label  \
0      December 9 , 1970 The economy of the Southeast...                1   
1      be dropping 300 employees by the end of the ye...                0   
2      percent above the year-ago level . This prompt...                1   
3      lowering of interest rates '' , '' improving c...                1   
4      November 11 , 1970 Our directors report a weak...                0   
...                                                  ...              ...   
29516  Beige Book Report : St Louis January 17 , 2024...                1   
29517  modestly since our previous report . Multiple ...                1   
29518  numbers in early 2024 . Manufacturing Manufact...                2   
29519  Rock metros . In the Louisville metro area , y...                1   
29520  expanded significantly compared with 2022 , re...                0   

      predicted_class  
0               Mixed  

In [ ]:
df_no_text = text_df.drop(columns=['text'])
#df_no_text.to_csv('sentiment_all_Beige_Book_chunks.csv', index=False)

,file_names,predicted_label,predicted_class
0,1970_at (1)_chunk_1.txt,1,Mixed
1,1970_at (1)_chunk_2.txt,0,Negative
2,1970_at (1)_chunk_3.txt,1,Mixed
3,1970_at (1)_chunk_4.txt,1,Mixed
4,1970_at (2)_chunk_1.txt,0,Negative
...,...,...,...
29516,2024_sl (8)_chunk_1.txt,1,Mixed
29517,2024_sl (8)_chunk_2.txt,1,Mixed
29518,2024_sl (8)_chunk_3.txt,2,Positive
29519,2024_sl (8)_chunk_4.txt,1,Mixed


In [17]:
# Aggregate by month

# Step 1: Extract Year and Report Number
df_no_text["Year"] = df_no_text["file_names"].str.extract(r"(\d{4})").astype(int)
df_no_text["Report_Number"] = df_no_text["file_names"].str.extract(r"\((\d+)\)").astype(int)

# Step 2: Map Report Numbers to Months
    # Got months from Fed website and put them in a spreadsheet
    # See C:\Users\MCOB PHD 14\Desktop\bbFinal\Notebooks\BB_publication_months.xlsx
def get_month(year, report_number):
    # Define mappings for report numbers to months
    if 1970 <= year <= 1978:
        month_mapping = {1: "12", 2: "11", 3: "10", 4: "09", 5: "08", 6: "07", 7: "06", 8: "05", 9: "04", 
                         10: "03", 11: "02", 12: "01"}
    elif year == 1979:
        month_mapping = {1: "11", 2: "10", 3: "09", 4: "08", 5: "07", 6: "05", 7: "04", 8: "03", 9: "01"}
    elif year == 1980:
        month_mapping = {1: "12", 2: "11", 3: "10", 4: "09", 5: "08", 6: "07", 7: "05", 8: "04", 9: "03", 10: "01"}
    elif year in {1981, 2017, 2023}:
        month_mapping = {1: "11", 2: "10", 3: "09", 4: "07", 5: "05", 6: "04", 7: "03", 8: "01"}
    elif year == 2012:
        month_mapping = {1: "11", 2: "10", 3: "08", 4: "07", 5: "06", 6: "04", 7: "02", 8: "01"}
    elif year in {2018, 2020, 2024}:
        month_mapping = {1: "12", 2: "10", 3: "09", 4: "07", 5: "05", 6: "04", 7: "03", 8: "01"}
    elif year in {2002, 2003, 2005, 2006, 2007, 2011, 2016, 2019, 2022}:
        month_mapping = {1: "11", 2: "10", 3: "09", 4: "07", 5: "06", 6: "04", 7: "03", 8: "01"}
    elif year in {2004, 2008, 2009, 2010, 2013, 2014, 2015, 2021}:
        month_mapping = {1: "12", 2: "10", 3: "09", 4: "07", 5: "06", 6: "04", 7: "03", 8: "01"}
    elif year in {1988, 2001}:
        month_mapping = {1: "11", 2: "10", 3: "09", 4: "08", 5: "06", 6: "05", 7: "03", 8: "01"}
    elif year == 1983:
        month_mapping = {1: "12", 2: "11", 3: "09", 4: "08", 5: "07", 6: "05", 7: "03", 8: "02"}
    elif year in {1984, 1985, 1986, 1987, 1990, 1991, 1996, 1997}:
        month_mapping = {1: "12", 2: "10", 3: "09", 4: "08", 5: "06", 6: "05", 7: "03", 8: "01"}
    else:
        month_mapping = {1: "12", 2: "11", 3: "09", 4: "08", 5: "06", 6: "05", 7: "03", 8: "01"}
    return month_mapping.get(report_number, "Unknown")

df_no_text["Month"] = df_no_text.apply(lambda row: get_month(row["Year"], row["Report_Number"]), axis=1)

# Step 3: Convert Year and Month to Date-Time Format
df_no_text["DATE"] = df_no_text.apply(lambda row: f"{row['Year']}-{row['Month']}-01", axis=1)
df_no_text["DATE"] = pd.to_datetime(df_no_text["DATE"], errors="coerce")

df_no_text.head()


,file_names,predicted_label,predicted_class,Year,Report_Number,Month,DATE
0,1970_at (1)_chunk_1.txt,1,Mixed,1970,1,12,1970-12-01
1,1970_at (1)_chunk_2.txt,0,Negative,1970,1,12,1970-12-01
2,1970_at (1)_chunk_3.txt,1,Mixed,1970,1,12,1970-12-01
3,1970_at (1)_chunk_4.txt,1,Mixed,1970,1,12,1970-12-01
4,1970_at (2)_chunk_1.txt,0,Negative,1970,2,11,1970-11-01


In [24]:
df_no_text.head()

,file_names,predicted_label,predicted_class,Year,Report_Number,Month,DATE
0,1970_at (1)_chunk_1.txt,1,Mixed,1970,1,12,1970-12-01
1,1970_at (1)_chunk_2.txt,0,Negative,1970,1,12,1970-12-01
2,1970_at (1)_chunk_3.txt,1,Mixed,1970,1,12,1970-12-01
3,1970_at (1)_chunk_4.txt,1,Mixed,1970,1,12,1970-12-01
4,1970_at (2)_chunk_1.txt,0,Negative,1970,2,11,1970-11-01


In [33]:
# Step 2: Aggregate counts by Year, Month, DATE, and predicted_class
aggregated = (
    df_no_text.groupby(['Year', 'Month', 'DATE', 'predicted_class'])
    .size()
    .reset_index(name='class_count')
)

# Step 3: Calculate total counts per month
total_counts = (
    df_no_text.groupby(['Year', 'Month', 'DATE'])
    .size()
    .reset_index(name='total_count')
)

# Step 4: Merge total counts with the sentiment class counts
merged = pd.merge(aggregated, total_counts, on=['Year', 'Month', 'DATE'])

# Step 5: Pivot table to create separate columns for each sentiment class
pivoted = merged.pivot_table(
    index=['Year', 'Month', 'DATE', 'total_count'],  # Include total_count in the pivot table
    columns='predicted_class',
    values='class_count',
    fill_value=0
).reset_index()

# Step 6: Rename columns for clarity
pivoted.rename(columns={
    'Negative': 'count_negative',
    'Positive': 'count_positive',
    'Mixed': 'count_mixed'
}, inplace=True)

# Step 7: Calculate fractions (proportions) of each sentiment class
pivoted['fraction_negative'] = pivoted['count_negative'] / pivoted['total_count']
pivoted['fraction_positive'] = pivoted['count_positive'] / pivoted['total_count']
pivoted['fraction_mixed'] = pivoted['count_mixed'] / pivoted['total_count']

# Step 8: Calculate the sentiment scores
# 1. Fraction of negative instances divided by fraction of positive instances
pivoted['negative_positive_ratio'] = (
    pivoted['fraction_negative'] / pivoted['fraction_positive']
)

# 2. Normalized Sentiment Index
pivoted['normalized_sentiment'] = (
    pivoted['fraction_positive'] - pivoted['fraction_negative']
) / (pivoted['fraction_positive'] + pivoted['fraction_negative'] + pivoted['fraction_mixed'])

# Handle potential division by zero (e.g., if there are no positive instances)
pivoted.replace([float('inf'), -float('inf')], float('nan'), inplace=True)

# Display the final DataFrame
pivoted


predicted_class,Year,Month,DATE,total_count,count_mixed,count_negative,count_positive,fraction_negative,fraction_positive,fraction_mixed,negative_positive_ratio,normalized_sentiment
0,1970,05,1970-05-01,42,10.0,30.0,2.0,0.714286,0.047619,0.238095,15.000000,-0.666667
1,1970,06,1970-06-01,49,12.0,31.0,6.0,0.632653,0.122449,0.244898,5.166667,-0.510204
2,1970,07,1970-07-01,45,16.0,27.0,2.0,0.600000,0.044444,0.355556,13.500000,-0.555556
3,1970,08,1970-08-01,43,15.0,24.0,4.0,0.558140,0.093023,0.348837,6.000000,-0.465116
4,1970,09,1970-09-01,42,18.0,13.0,11.0,0.309524,0.261905,0.428571,1.181818,-0.047619
...,...,...,...,...,...,...,...,...,...,...,...,...
465,2023,11,2023-11-01,74,35.0,32.0,7.0,0.432432,0.094595,0.472973,4.571429,-0.337838
466,2024,01,2024-01-01,74,36.0,18.0,20.0,0.243243,0.270270,0.486486,0.900000,0.027027
467,2024,03,2024-03-01,75,46.0,13.0,16.0,0.173333,0.213333,0.613333,0.812500,0.040000
468,2024,04,2024-04-01,78,44.0,12.0,22.0,0.153846,0.282051,0.564103,0.545455,0.128205


In [41]:
# Drop predicted_class, Year, Month
final_df = pivoted.drop(columns=['Year', 'Month'])

# Save the final DataFrame to a CSV file
final_df.to_csv('sentiment_all_Beige_Book_chunks.csv', index=False)


In [42]:
final_df.describe()

predicted_class,DATE,total_count,count_mixed,count_negative,count_positive,fraction_negative,fraction_positive,fraction_mixed,negative_positive_ratio,normalized_sentiment
count,470,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,470.000000,468.000000,470.000000
mean,1995-08-24 14:27:03.829787264,62.810638,24.065957,12.861702,25.882979,0.212170,0.408085,0.379744,2.121734,0.195915
min,1970-05-01 00:00:00,40.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.000000,-0.805195
25%,1980-08-08 18:00:00,53.000000,17.000000,5.000000,13.250000,0.084746,0.245283,0.303628,0.153263,-0.045995
50%,1995-01-30 12:00:00,60.000000,22.000000,9.000000,26.000000,0.150342,0.438262,0.370370,0.348913,0.284367
75%,2009-09-23 12:00:00,74.000000,29.000000,17.000000,36.750000,0.285714,0.581208,0.460526,1.181818,0.490296
max,2024-05-01 00:00:00,90.000000,50.000000,63.000000,62.000000,0.818182,0.847222,0.666667,63.000000,0.805556
std,NaN,12.272505,9.113002,11.222124,14.972652,0.179236,0.208894,0.108998,6.805527,0.373689
